In [8]:
import sys, cv2
import numpy as np
#import tracktor_revised as tr
import matplotlib.pyplot as plt
import pickle
import os
from os.path import join
import collections

from importlib import reload
import platform
import re
import datetime
import multiprocessing
from glob import glob
import logging
from tracker.tank import Tank
from tracker.tracker import Tracker
from tracker import utils

In [2]:
# with open('partialFish.pkl','rb') as f:
#     fish = pickle.load(f)

# # with open('sortedFish.pik','rb') as f:
# #     sortedFish = pickle.load(f)

# # posU=sortedFish[0]
# # nonOverlappingRange=sortedFish[1]
# # fishU=sortedFish[2]

In [3]:
''' Select input files to track and an output directory for each. '''

tracking_dir = 'symposium-output'

# input_files = sorted(glob('../Fish_Tracking_Data/'))
input_files = ['../Fish_Tracking_Data/SF_n2_t2_3113.mp4']

input_dict = {}
for input_file in input_files:
    d,fn       = os.path.split(input_file)
    fn,ext     = os.path.splitext(fn)
    output_dir = os.path.join(tracking_dir,fn)
    n_ind      = int(re.findall('_n(\d+)[^\d]*_',fn)[0])
    input_dict[input_file] = dict( output_dir=output_dir, n_ind=n_ind )

for f,D in input_dict.items():
    print(f)
    print('   ',D)

../Fish_Tracking_Data/SF_n2_t2_3113.mp4
    {'output_dir': 'symposium-output/SF_n2_t2_3113', 'n_ind': 2}


In [4]:
''' Create output directories. Trace tanks. '''

for input_file,D in input_dict.items():
    tracker = Tracker(input_video=input_file, **D)
    
    # Create output directory.
    tracker.init_directory()
    
    # Create a link to the input video in the output directory.
    input_shortcut = os.path.join(output_dir,'raw'+ext)
    if not 'windows' in platform.system().lower() and not os.path.exists(input_shortcut):
        os.symlink(os.path.relpath(input_file,output_dir),input_shortcut)
        
    # Trace or load tank.
    tracker.init_tank()

In [5]:
settings = dict(
    t_start          = 0,     # Time at which to start tracking, in seconds.
    t_end            = 3,    # Time at which to end tracking, in seconds.
    
    # Contour detection.
    n_pixel_blur     =  7,    # square-root of n-pixels for threshold blurring
    block_size       = 15,    # contour block size
    threshold_offset = 13,    # threshold offset for contour-finding
    min_area         = 50,    # minimum area for detection
    max_area         = 400,   # maximum area for detection
    ideal_area       = 200,   # ideal area to rank contours in first frame (default=(min_area+max_area)/2)
    max_aspect       = 20,    # maximum aspect ratio for detection
    ideal_aspect     = 3,     # ideal aspect ratio to rank contours in first frame (default=max_aspect/2)
    area_penalty     = 0.5,   # weight of area change when connecting fish across frames
    reversal_threshold = 0.5, # average frame-to-frame displacement against the director 
                            # over the last few frames to trigger a reversal. 
    live_preview     = False, # Toggle live preview of tracking.

    # Background subtraction (for naive background subtraction only).
    bkgSub_options   = dict( n_training_frames = 100, # number of frames used to compute background
                             t_start = 0, t_end = -1, # time range used to compute background
                             contrast_factor = 4 ),   # post-subtraction contrast enhancement factor
    
    # What information to draw on the tracking output video.
    video_output_options = dict( tank=True, points=False, directors=True, timestamp=True, 
                                 contours=True, contour_color=(100,255,0), contour_thickness=1 )
    )

In [57]:
globals().update(settings)
globals().update(input_dict[input_file])
video_output_options = settings['video_output_options']

utils.add_log_file(os.path.join(output_dir,'log.txt'))
# logging.info(utils.parindent+'Initializing')
tracker = Tracker( input_video=input_file, **input_dict[input_file], **settings )
tracker.init_all()
tracker.save_settings()
print()

frames_dir = os.path.join(output_dir,'frames')
if not os.path.exists(frames_dir):
    os.mkdir(frames_dir)

#frame_list = range(5000,tracker.n_frames,1)
# frame_list = range(5150,5200,1) # 2 simple swaps in succession
# frame_list = range(5000,5150,1)
frame_list = range(5190,5211,1)
# frame_list = range(5230,5250,1)
tracker.set_frame(frame_list[0])
contourList=[] #!
fish=[]
has_overlapped = False
for i_frame in frame_list:
    print(' '*100+'\r'+f'{i_frame}',end='\r')
    #logging.info(i_frame)
    tracker.get_next_frame()
    
    frame = tracker.frame.copy()
    
    tracker.subtract_background()
    tracker.mask_tank()
    tracker.detect_contours()
    tracker.connect_frames()    
    
    n = len(tracker.contours)
    if n==1:
        cv2.drawContours(frame, tracker.contours, 0, (255,0,255), -1)
        has_overlapped = True
        cv2.imwrite(join(frames_dir,f'{i_frame}.png'),frame)
    if n==2 and not has_overlapped:
        
#         pos = tracker.data[-2:,:,:2]
#         if pos.shape[0]>1:
# #             d1 = np.hypot(*pos[-2,0]-pos[-1,0])+np.hypot(*pos[-2,1]-pos[-1,1])
# #             d2 = np.hypot(*pos[-2,0]-pos[-1,1])+np.hypot(*pos[-2,1]-pos[-1,0])
#             d1 = np.hypot(*pos[-2,0]-pos[-1,0])
#             d2 = np.hypot(*pos[-2,0]-pos[-1,1])
#             if d2<d1:
#                 print('unswap?')
#                 tracker.contours = tracker.contours[::-1]
        
        colors = [ (0,0,255), (255,0,0) ]
        for i in range(n):
            cv2.drawContours(frame, tracker.contours, i, colors[i], -1)
    
#     cv2.imwrite(join(frames_dir,f'{i_frame}.png'),frame)
    
    tracker.frame[:] = frame[:]
    for i in range(10):
        tracker.write_frame()    
    
    #break
tracker.release()
#tracker.save_trial()

with open('partialFish.pkl','wb') as fh:
     pickle.dump(fish,fh)

INFO:root:Tank loaded from symposium-output/SF_n2_t2_3113/tank.pik


INFO:root:Video capture released
